## Inferencing SpeciesNet Model

In [ ]:
# Download the dataset from Google drive
url_path = 'https://drive.google.com/drive/u/0/folders/1R17xoqkLWiFjHCCftBO_Gvtofu_5Hsjc'
!gdown --id 1R17xoqkLWiFjHCCftBO_Gvtofu_5Hsjc --folder -O data

In [ ]:
# Download the model
!curl -L -o model.tar.gz  https://www.kaggle.com/api/v1/models/google/speciesnet/keras/v4.0.0a/3/download

!tar -xzf model.tar.gz -C model

In [ ]:
# Run the model
!python -m speciesnet.scripts.run_model --folders "data"   --predictions_json "impala_malawi.json"   --model "/model"


In [ ]:
import json
with open('impala_malawi.json') as f:
  output_file = json.load(f)

In [ ]:
import pandas as pd
output_df = pd.json_normalize(output_file['predictions'])

In [ ]:
data = [output_df['prediction'].iloc[i].split(';')[1:] for i in range(len(output_df))]

columns = ['Class', 'Order', 'Family', 'Genus', 'Species', 'Common Name']
df = pd.DataFrame(data, columns=columns)

# replace the '' with Unknown
df = df.replace('', 'Unknown')

df_counts = df.groupby(columns).size().reset_index(name='Count')
df_counts.fillna('Unknown', inplace=True)

In [ ]:
import plotly.express as px

fig = px.treemap(
    df_counts,
    path=columns,
    values='Count',
    title='Hierarchical Animal Classification Treemap'
)
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.show()


## Inferencing MegaDetector

In [ ]:
'''
add os walk to the nested directories to get the outputsas a method where if enabled the 
walk happens if not just that repository to get the images
'''

In [ ]:
from ultralytics import YOLO
import os


model_path = os.path.join(os.getcwd(), 'models', 'MDV6b-yolov9c.pt')

# Load the yolo model
model = YOLO(model_path)


img_list = [os.path.join('data', img) for img in os.listdir('data') if img.endswith('.jpg')]

# Run the tracking and inferencing of the clip
results = model.track(img_list, save=True, save_txt=True, conf=0.5, iou=0.7)

## Inferencing MegaDetector with SAHI for overhead processing

- Meant to run on Drones
- Tiling the image to windows

In [ ]:
!pip install -U torch sahi ultralytics
!pip install ipywidgets

In [ ]:
!pip install -U PytorchWildlife


In [ ]:
from PytorchWildlife.models import detection as pw_detection

In [ ]:
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image

In [ ]:
# Load the MegaDetector V6 model
detection_model = pw_detection.MegaDetectorV6(device='cuda', 
                                              pretrained=True, 
                                              version="MDV6-yolov9-e") # Model loaded from /root/.cache/torch/hub/checkpoints

In [ ]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path="/root/.cache/torch/hub/checkpoints/MDV6-yolov9-e-1280.pt", # any yolov8/yolov9/yolo11/yolo12/rt-detr det model is supported
    confidence_threshold=0.5,
    device="cuda:0", # or 'cuda:0' if GPU is available
)

In [ ]:
result = get_sliced_prediction(
    "/content/drive/MyDrive/Aerial Widlife Photos/EWB Tsavo survey Left observer/L 02-25-2014/IMG_8730.JPG",
    detection_model,
    slice_height = 512,
    slice_width = 512,
    overlap_height_ratio = 0.8,
    overlap_width_ratio = 0.8
)

In [ ]:
result.export_visuals(export_dir="demo_data/", hide_conf=True, hide_labels=True)

Image("demo_data/prediction_visual.png")